In [13]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## 라이브러리 불러오기

In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from pprint import pprint

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
np.set_printoptions(threshold = None, linewidth = 200)

print(tf.__version__)
print(keras.__version__)

1.13.1
2.2.4-tf


## Eager 모드 활성화

In [15]:
tf.enable_eager_execution()

# sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15683312916253198321
]


## Hyper parameters

In [16]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

# tf.set_random_seed(777)

## Creating a Checkpoint Directory

In [17]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'yolo_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## Data

In [18]:
import json
# mnist = keras.datasets.mnist
class_names = ['Car']

xwidth = 1280
xheight = 720

boxfile = open("highway-cctv-footage-images/ground_truth_boxes/labelbox.json", "r")#, encoding = "UTF-8")
boxes = json.load(boxfile)
# boxes = [box[:-1] for box in boxes]
for i in range(len(boxes)):
#     pprint(boxes[i]['External ID'])
    pprint(boxes[i]['Label']['Car'])

[{'geometry': [{'x': 887, 'y': 447},
               {'x': 887, 'y': 629},
               {'x': 1074, 'y': 629},
               {'x': 1074, 'y': 447}]},
 {'geometry': [{'x': 142, 'y': 54},
               {'x': 142, 'y': 85},
               {'x': 200, 'y': 85},
               {'x': 200, 'y': 54}]},
 {'geometry': [{'x': 319, 'y': 32},
               {'x': 319, 'y': 65},
               {'x': 362, 'y': 65},
               {'x': 362, 'y': 32}]},
 {'geometry': [{'x': 286, 'y': 13},
               {'x': 286, 'y': 38},
               {'x': 326, 'y': 38},
               {'x': 326, 'y': 13}]},
 {'geometry': [{'x': 578, 'y': 69},
               {'x': 578, 'y': 98},
               {'x': 616, 'y': 98},
               {'x': 616, 'y': 69}]},
 {'geometry': [{'x': 783, 'y': 68},
               {'x': 783, 'y': 96},
               {'x': 822, 'y': 96},
               {'x': 822, 'y': 68}]},
 {'geometry': [{'x': 747, 'y': 37},
               {'x': 747, 'y': 68},
               {'x': 780, 'y': 68},
          

 {'geometry': [{'x': 764, 'y': 80},
               {'x': 764, 'y': 116},
               {'x': 808, 'y': 116},
               {'x': 808, 'y': 80}]},
 {'geometry': [{'x': 651, 'y': 70},
               {'x': 651, 'y': 102},
               {'x': 690, 'y': 102},
               {'x': 690, 'y': 70}]},
 {'geometry': [{'x': 586, 'y': 49},
               {'x': 586, 'y': 84},
               {'x': 627, 'y': 84},
               {'x': 627, 'y': 49}]},
 {'geometry': [{'x': 421, 'y': 15},
               {'x': 421, 'y': 44},
               {'x': 451, 'y': 44},
               {'x': 451, 'y': 15}]}]
[{'geometry': [{'x': 641, 'y': 127},
               {'x': 641, 'y': 175},
               {'x': 689, 'y': 175},
               {'x': 689, 'y': 127}]},
 {'geometry': [{'x': 786, 'y': 186},
               {'x': 786, 'y': 280},
               {'x': 887, 'y': 280},
               {'x': 887, 'y': 186}]},
 {'geometry': [{'x': 385, 'y': 25},
               {'x': 385, 'y': 65},
               {'x': 426, 'y': 65},
    

 {'geometry': [{'x': 649, 'y': 70},
               {'x': 649, 'y': 100},
               {'x': 685, 'y': 100},
               {'x': 685, 'y': 70}]}]
[{'geometry': [{'x': 929, 'y': 239},
               {'x': 929, 'y': 317},
               {'x': 1028, 'y': 317},
               {'x': 1028, 'y': 239}]},
 {'geometry': [{'x': 747, 'y': 128},
               {'x': 747, 'y': 179},
               {'x': 805, 'y': 179},
               {'x': 805, 'y': 128}]},
 {'geometry': [{'x': 786, 'y': 61},
               {'x': 786, 'y': 91},
               {'x': 819, 'y': 91},
               {'x': 819, 'y': 61}]},
 {'geometry': [{'x': 707, 'y': 33},
               {'x': 707, 'y': 62},
               {'x': 746, 'y': 62},
               {'x': 746, 'y': 33}]},
 {'geometry': [{'x': 652, 'y': 53},
               {'x': 652, 'y': 75},
               {'x': 682, 'y': 75},
               {'x': 682, 'y': 53}]},
 {'geometry': [{'x': 123, 'y': 46},
               {'x': 123, 'y': 107},
               {'x': 214, 'y': 107},
  

 {'geometry': [{'x': 364, 'y': 32},
               {'x': 364, 'y': 71},
               {'x': 406, 'y': 71},
               {'x': 406, 'y': 32}]},
 {'geometry': [{'x': 143, 'y': 46},
               {'x': 143, 'y': 83},
               {'x': 206, 'y': 83},
               {'x': 206, 'y': 46}]}]
[{'geometry': [{'x': 503, 'y': 709},
               {'x': 292, 'y': 709},
               {'x': 292, 'y': 484},
               {'x': 503, 'y': 484}]},
 {'geometry': [{'x': 773, 'y': 169},
               {'x': 773, 'y': 245},
               {'x': 855, 'y': 245},
               {'x': 855, 'y': 169}]},
 {'geometry': [{'x': 742, 'y': 45},
               {'x': 742, 'y': 73},
               {'x': 775, 'y': 73},
               {'x': 775, 'y': 45}]},
 {'geometry': [{'x': 307, 'y': 51},
               {'x': 307, 'y': 94},
               {'x': 369, 'y': 94},
               {'x': 369, 'y': 51}]},
 {'geometry': [{'x': 42, 'y': 73},
               {'x': 42, 'y': 115},
               {'x': 123, 'y': 115},
        

               {'x': 232, 'y': 40}]},
 {'geometry': [{'x': 248, 'y': 517},
               {'x': 248, 'y': 720},
               {'x': 471, 'y': 720},
               {'x': 471, 'y': 517}]},
 {'geometry': [{'x': 641, 'y': 214},
               {'x': 641, 'y': 336},
               {'x': 734, 'y': 336},
               {'x': 734, 'y': 214}]},
 {'geometry': [{'x': 761, 'y': 174},
               {'x': 761, 'y': 230},
               {'x': 841, 'y': 230},
               {'x': 841, 'y': 174}]},
 {'geometry': [{'x': 645, 'y': 119},
               {'x': 645, 'y': 164},
               {'x': 701, 'y': 164},
               {'x': 701, 'y': 119}]},
 {'geometry': [{'x': 536, 'y': 165},
               {'x': 536, 'y': 219},
               {'x': 604, 'y': 219},
               {'x': 604, 'y': 165}]},
 {'geometry': [{'x': 335, 'y': 34},
               {'x': 335, 'y': 58},
               {'x': 371, 'y': 58},
               {'x': 371, 'y': 34}]}]
[{'geometry': [{'x': 425, 'y': 298},
               {'x': 425, 'y'

## Datasets

In [19]:
# Input Images
num_images = 98
image = np.zeros((num_images, 720, 1280, 3))
for i in range(num_images):
    image[i, :, :, :] = cv2.imread("highway-cctv-footage-images/input_images/img_{}.jpg".format(str(i)), cv2.IMREAD_UNCHANGED)
print(image.shape)

# Output Bounding Boxes Informations


(98, 720, 1280, 3)


## Model

In [22]:
def create_model():
    inputs = keras.Input(shape=(720, 1280, 3))
    conv1 = keras.layers.Conv2D(filters=64, kernel_size=7, activation=tf.nn.relu, padding='SAME')(inputs)
    pool1 = keras.layers.MaxPool2D(padding='SAME')(conv1)
    conv2 = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME')(pool1)
    pool2 = keras.layers.MaxPool2D(padding='SAME')(conv2)
    conv3 = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME')(pool2)
    pool3 = keras.layers.MaxPool2D(padding='SAME')(conv3)
    conv4 = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME')(pool3)
    pool4 = keras.layers.MaxPool2D(padding='SAME')(conv4)
    conv5 = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME')(pool4)
    pool5 = keras.layers.MaxPool2D(pool_size = (5,5), padding='SAME')(conv5)
    pool5_flat = keras.layers.Flatten()(pool5)
    dense1 = keras.layers.Dense(units = 256, activation=tf.nn.relu)(pool5_flat)
    drop1 = keras.layers.Dropout(0.4)(dense1)
    dense2_1 = keras.layers.Dense(16 * 9 * 5, activation=tf.nn.relu)(drop1)
    dense2_2 = keras.layers.Dense(16 * 9 * 1, activation=tf.nn.sigmoid)(drop1)
    resh1_1 = keras.layers.Reshape((9, 16, 5))(dense2_1)
    resh1_2 = keras.layers.Reshape((9, 16, 1))(dense2_2)
    outputs = keras.layers.Concatenate(axis=-1)([resh1_1, resh1_2])
    return keras.Model(inputs = inputs, outputs = outputs)

model = create_model()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 720, 1280, 3) 0                                            
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 720, 1280, 64 9472        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_15 (MaxPooling2D) (None, 360, 640, 64) 0           conv2d_15[0][0]                  
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 360, 640, 64) 36928       max_pooling2d_15[0][0]           
__________________________________________________________________________________________________
max_poolin

## Loss function

In [10]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=labels))    
    return loss   

## Gradient 계산

In [11]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## 모델 정확도 계산

In [12]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## Optimizer

In [13]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

## Checkpoint

In [14]:
checkpoint = tf.train.Checkpoint(cnn=model)

## Training

In [15]:
# train my model
import time

print('Learning started. It takes sometime.')
s_time = time.time()
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))
    
    checkpoint.save(file_prefix=checkpoint_prefix)

e_time = time.time() - s_time
print('Learning Finished!')
print('{:5f} seconds'.format(e_time))

Learning started. It takes sometime.
Epoch: 1 loss = 0.17266375 train accuracy =  0.9590 test accuracy =  0.9840
Epoch: 3 loss = 0.03135390 train accuracy =  0.9936 test accuracy =  0.9922
Epoch: 4 loss = 0.02327727 train accuracy =  0.9954 test accuracy =  0.9930
Epoch: 5 loss = 0.01839644 train accuracy =  0.9967 test accuracy =  0.9935
Epoch: 6 loss = 0.01527054 train accuracy =  0.9974 test accuracy =  0.9925
Epoch: 7 loss = 0.01112090 train accuracy =  0.9982 test accuracy =  0.9921
Epoch: 8 loss = 0.01158327 train accuracy =  0.9983 test accuracy =  0.9901
Epoch: 9 loss = 0.00984489 train accuracy =  0.9986 test accuracy =  0.9928
Epoch: 10 loss = 0.00841346 train accuracy =  0.9991 test accuracy =  0.9929
Epoch: 11 loss = 0.00715333 train accuracy =  0.9992 test accuracy =  0.9926
Epoch: 12 loss = 0.00622725 train accuracy =  0.9992 test accuracy =  0.9939
Epoch: 13 loss = 0.00546163 train accuracy =  0.9993 test accuracy =  0.9922
Epoch: 14 loss = 0.00527327 train accuracy =  0